In [25]:
# Packages

from gurobipy import Model, GRB, quicksum
import numpy as np
from itertools import product
import pandas as pd

Pattern Generation

In [26]:
n_teams = 6
n_minus_1 = n_teams - 1
opt_1 = 3
opt_2 = 2

def valid_pattern(pattern):
    # Count occurrences of 'H' and 'A'
    h_count = pattern.count('H')
    a_count = pattern.count('A')

    # Check for similar number of H and A matches (4H-5A or 5H-4A)
    if not (h_count == opt_1 and a_count == opt_2 or h_count == opt_2 and a_count == opt_1):
        return False

    # Check for more than two consecutive H or A matches
    for i in range(len(pattern) - 2):
        if pattern[i] == pattern[i + 1] == pattern[i + 2]:
            return False

    return True

# Generate all combinations of H and A for N-1 matches
all_patterns = product('HA', repeat=n_minus_1)

# Filter patterns that meet the criteria
valid_patterns = [''.join(pattern) for pattern in all_patterns if valid_pattern(pattern)]

# Output the number of valid patterns
print(f"Total valid patterns: {len(valid_patterns)}")


Total valid patterns: 14


Defining the Set of Complimentary and Feasible Patterns

In [27]:
# Function to check if two patterns are complementary
def are_complementary(pattern1, pattern2):
    return all((c1 == 'H' and c2 == 'A') or (c1 == 'A' and c2 == 'H') for c1, c2 in zip(pattern1, pattern2))

# Find all pairs of complementary patterns
C = []
for i in range(len(valid_patterns)):
    for j in range(i + 1, len(valid_patterns)):
        if are_complementary(valid_patterns[i], valid_patterns[j]):
            C.append((valid_patterns[i], valid_patterns[j]))

print(C)

C_indices = [(valid_patterns.index(x), valid_patterns.index(y)) for x, y in C]

print(C_indices)


[('HHAHA', 'AAHAH'), ('HHAAH', 'AAHHA'), ('HAHHA', 'AHAAH'), ('HAHAH', 'AHAHA'), ('HAHAA', 'AHAHH'), ('HAAHH', 'AHHAA'), ('HAAHA', 'AHHAH')]
[(0, 13), (1, 12), (2, 11), (3, 10), (4, 9), (5, 8), (6, 7)]


In [28]:
len(C_indices)

7

In [29]:
# Dictionary `feasible_opponents` where feasible_opponents[i, k]
# is the set of feasible opponents for pattern i in week t


# Initialize the feasible opponents dictionary
feasible_opponents = {}

# Iterate through each pattern and week
num_patterns = len(valid_patterns)
num_weeks = len(valid_patterns[0])

for i in range(num_patterns):
    for k in range(num_weeks):
        # Get the H/A status of pattern i in week k
        current_status = valid_patterns[i][k]
        
        # Find all patterns that have the opposite status in week k
        feasible_opponents[i, k] = {j for j in range(num_patterns) if j != i and valid_patterns[j][k] != current_status}


# Print the feasible opponents dictionary for verification
for key, value in feasible_opponents.items():
    print(f"Pattern {key[0]} in week {key[1]} can play against patterns {value}")


Pattern 0 in week 0 can play against patterns {7, 8, 9, 10, 11, 12, 13}
Pattern 0 in week 1 can play against patterns {2, 3, 4, 5, 6, 12, 13}
Pattern 0 in week 2 can play against patterns {2, 3, 4, 7, 8, 12, 13}
Pattern 0 in week 3 can play against patterns {1, 3, 4, 7, 8, 11, 13}
Pattern 0 in week 4 can play against patterns {1, 3, 5, 7, 9, 11, 13}
Pattern 1 in week 0 can play against patterns {7, 8, 9, 10, 11, 12, 13}
Pattern 1 in week 1 can play against patterns {2, 3, 4, 5, 6, 12, 13}
Pattern 1 in week 2 can play against patterns {2, 3, 4, 7, 8, 12, 13}
Pattern 1 in week 3 can play against patterns {0, 2, 5, 6, 9, 10, 12}
Pattern 1 in week 4 can play against patterns {0, 2, 4, 6, 8, 10, 12}
Pattern 2 in week 0 can play against patterns {7, 8, 9, 10, 11, 12, 13}
Pattern 2 in week 1 can play against patterns {0, 1, 7, 8, 9, 10, 11}
Pattern 2 in week 2 can play against patterns {0, 1, 5, 6, 9, 10, 11}
Pattern 2 in week 3 can play against patterns {1, 3, 4, 7, 8, 11, 13}
Pattern 2 in w

In [30]:
# Create the list of teams and ranking
teams = ['Team 1', 'Team 2', 'Team 3', 'Team 4', 'Team 5', 'Team 6']

ranking = {
    1: teams[:3],  # Category 1
    2: teams[3:6],  # Category 2
    3: teams[6:]    # Category 3
}

# Assign a unique number to each team
team_numbers = {i : team for i, team in enumerate(teams)}

# Create a reverse mapping to look up the number of a team
team_to_number = {team: number for number, team in team_numbers.items()}

# Create a dictionary to store subsets of stronger teams
stronger_teams = {}

# Iterate through each team and determine stronger teams
for position, teams_at_position in ranking.items():
    for team in teams_at_position:
        team_number = team_to_number[team]  # Get the team's unique number
        stronger_teams[team_number] = [
            team_to_number[stronger_team]
            for higher_position in range(1, position)  # Look at higher positions only
            for stronger_team in ranking[higher_position]
        ]

# Set of teams located in the same town
D = [(0,5)]

# Print the team-to-number mapping and the stronger teams dictionary
print("Team Numbers:", team_numbers)
print("Stronger Teams:", stronger_teams)
print('Pairs of teams located in the same town', D)

Team Numbers: {0: 'Team 1', 1: 'Team 2', 2: 'Team 3', 3: 'Team 4', 4: 'Team 5', 5: 'Team 6'}
Stronger Teams: {0: [], 1: [], 2: [], 3: [0, 1, 2], 4: [0, 1, 2], 5: [0, 1, 2]}
Pairs of teams located in the same town [(0, 5)]


## Model


In [31]:
from gurobipy import Model, GRB, quicksum

# Initialize the model
m = Model("Sports_Big")
m.reset()


# Sets and Parameters

T = range(n_teams)              # Team indices 
W = range(n_minus_1)              # Week indices 
W_minus_1 = range(n_minus_1 - 1)      # For consecutive weeks 
P = range(len(valid_patterns))  # Pattern indices 


# Decision Variables

# r_var[p, pp, w] = 1 if pattern p is paired with pattern pp in week w
r_var = m.addVars(P, P, W, vtype=GRB.BINARY, name="r")

# y_var[p] = 1 if pattern p is selected as part of the 10 patterns
y_var = m.addVars(P, vtype=GRB.BINARY, name="y")

# z_var[i, p] = 1 if team i is assigned to pattern p
z_var = m.addVars(T, P, vtype=GRB.BINARY, name="z")

# s_var[i, w] = 1 if team i incurs a carry-over effect between week w and w+1
s_var = m.addVars(T, W_minus_1, vtype=GRB.BINARY, name="s")

# x_var[i, j, w] = 1 if team i plays team j in week w
x_var = m.addVars(T, T, W, vtype=GRB.BINARY, name="x")


# Select exactly N patterns to be used consistently across all weeks
m.addConstr(quicksum(y_var[p] for p in P) == n_teams, name="select_10_patterns")


# Ensure each pattern used in a week is one of the N selected patterns.
# For each pattern p and week w, the sum over valid opponents equals y_var[p].

for p in P:
    for w in W:
        if (p, w) in feasible_opponents:
            m.addConstr(
                quicksum(r_var[p, pp, w] for pp in feasible_opponents[(p, w)] if p < pp) +
                quicksum(r_var[pp, p, w] for pp in feasible_opponents[(p, w)] if pp < p)
                == y_var[p],
                name=f"pattern_consistency_{p}_{w}"
            )


# Each pair of selected patterns is selected for at most one week.
for p in P:
    for pp in P:
        if p != pp:
            m.addConstr(
                quicksum(r_var[p, pp, w] for w in W) <= y_var[p],
                name=f"B3_unique_match_{p}_{pp}"
            )


# Setting irrelevant r_var variables to 0:
# (a) If p >= pp
# (b) If pp is not a feasible opponent for pattern p in week w.

for p in P:
    for pp in P:
        if p >= pp:
            for w in W:
                r_var[p, pp, w].ub = 0

for p in P:
    for w in W:
        for pp in P:
            if p < pp:
                if pp not in feasible_opponents.get((p, w), []):
                    r_var[p, pp, w].ub = 0


# Each team must be assigned to exactly one pattern.

for i in T:
    m.addConstr(
        quicksum(z_var[i, p] for p in P) == 1,
        name=f"team_pattern_allocation_{i}"
    )


# Each selected pattern must be assigned to exactly one team.

for p in P:
    m.addConstr(
        quicksum(z_var[i, p] for i in T) == y_var[p],
        name=f"team_pattern_unique_allocation_{p}"
    )


# Defining the x_var[i, j, w] variable.
# For each week, for each team pair (i, j) with i < j, and for each pattern pair (p, phi_prime) with p < phi_prime,
# enforce:
#   x_var[i,j,w] >= z_var[i,p] + z_var[i,phi_prime] + z_var[j,p] + z_var[j,phi_prime] + r_var[p, phi_prime, w] + r_var[phi_prime, p, w] - 2

for w in W:
    for i in T:
        for j in T:
            if i < j:
                for p in P:
                    for phi_prime in P:
                        if p < phi_prime:
                            m.addConstr(
                                x_var[i, j, w] >= z_var[i, p] + z_var[i, phi_prime] +
                                                   z_var[j, p] + z_var[j, phi_prime] +
                                                   r_var[p, phi_prime, w] + r_var[phi_prime, p, w] - 2,
                                name=f"match_constraint_link_{i}_{j}_{w}_{p}_{phi_prime}"
                            )


# Each team plays against every other team exactly once throughout the season.

for i in T:
    for j in T:
        if i < j:
            m.addConstr(
                quicksum(x_var[i, j, w] for w in W) == 1,
                name=f"one_match_{i}_{j}"
            )


# Each team plays exactly one match in any given week.

for w in W:
    for i in T:
        m.addConstr(
            quicksum(x_var[i, j, w] for j in T if i < j) +
            quicksum(x_var[j, i, w] for j in T if j < i)
            == 1,
            name=f"one_match_per_week_{i}_{w}"
        )


# Setting irrelevant x_var variables to 0 if i >= j.

for i in T:
    for j in T:
        if i >= j:
            for w in W:
                x_var[i, j, w].ub = 0


# Assignment of complementary patterns (same-venue constraint).
# For each team pair (i, j) in D and each complementary pair (c, c_prime) in C_indices,
# enforce: (z_var[i, c] + z_var[i, c_prime]) - (z_var[j, c_prime] + z_var[j, c]) = 0.

for (i, j) in D:
    for (c, c_prime) in C_indices:
        if c in P:
            m.addConstr(
                (z_var[i, c] + z_var[i, c_prime]) - (z_var[j, c_prime] + z_var[j, c]) == 0,
                name=f"comp_pattern_{i}_{j}_{c}_{c_prime}"
            )


# Carry-over effect between consecutive weeks.
# For each team i and each week w in W_minus_1, enforce:
#   sum_{j in stronger_teams[i]} [x_var[i, j, w] (if i<j) + x_var[j, i, w] (if j<i)] +
#   sum_{j in stronger_teams[i]} [x_var[i, j, w+1] (if i<j) + x_var[j, i, w+1] (if j<i)]
#   <= 1 + s_var[i, w],
# with corresponding lower-bound constraints.

for i in T:
    for w in W_minus_1:
        m.addConstr(
            quicksum(x_var[i, j, w] for j in stronger_teams[i] if i < j) +
            quicksum(x_var[j, i, w] for j in stronger_teams[i] if j < i) +
            quicksum(x_var[i, j, w+1] for j in stronger_teams[i] if i < j) +
            quicksum(x_var[j, i, w+1] for j in stronger_teams[i] if j < i)
            <= 1 + s_var[i, w],
            name=f"carry_over_effect_{i}_{w}"
        )
        m.addConstr(
            quicksum(x_var[i, j, w] for j in stronger_teams[i] if i < j) +
            quicksum(x_var[j, i, w] for j in stronger_teams[i] if j < i)
            >= s_var[i, w],
            name=f"carry_over_lower1_{i}_{w}"
        )
        m.addConstr(
            quicksum(x_var[i, j, w+1] for j in stronger_teams[i] if i < j) +
            quicksum(x_var[j, i, w+1] for j in stronger_teams[i] if j < i)
            >= s_var[i, w],
            name=f"carry_over_lower2_{i}_{w}"
        )


# Objective: Minimize total carry-over effect.

m.setObjective(quicksum(s_var[i, w] for i in T for w in W_minus_1), GRB.MINIMIZE)

m.update()

print("Number of constraints:", m.NumConstrs)
print("Number of variables:", m.NumVars)

# Solve the model
m.optimize()

if m.status == GRB.OPTIMAL:
    print("Optimal solution found")
    y_sol = {p: y_var[p].x for p in P if y_var[p].x > 0.5}
    z_sol = {(i, p): z_var[i, p].x for i in T for p in P if z_var[i, p].x > 0.5}
    s_sol = {(i, w): s_var[i, w].x for i in T for w in W_minus_1 if s_var[i, w].x > 0.5}
    x_sol = {(i, j, w): x_var[i, j, w].x for i in T for j in T for w in W if x_var[i, j, w].x > 0.5}
    r_sol = {(p, pp, w): r_var[p, pp, w].x for p in P for pp in P for w in W if r_var[p, pp, w].x > 0.5}
    print("Team-to-pattern allocation:", z_sol)
    print("Carry-over effect:", s_sol)
    print("Matches between teams:", x_sol)
    print("Pattern pairings:", r_sol)
    print("Selected patterns:", y_sol)
else:
    print("No optimal solution found")


Discarded solution information
Number of constraints: 7222
Number of variables: 1282
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 7222 rows, 1282 columns and 50092 nonzeros
Model fingerprint: 0xbda9ebcb
Variable types: 0 continuous, 1282 integer (1282 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Presolve removed 3509 rows and 866 columns
Presolve time: 0.06s
Presolved: 3713 rows, 416 columns, 22175 nonzeros
Variable types: 0 continuous, 416 integer (416 binary)
Found heuristic solution: objective 2.0000000

Root relaxation: objective 0.000000e+00, 205 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bou

Displaying Pattern Pairings

In [32]:

# Extract weeks dynamically
weeks = sorted({t for _, _, t in r_sol.keys()})
calendar_data = {f"Week {t+1}": [] for t in weeks}

# Populate calendar data for each week 
for t in weeks:
    week_pairs = []
    for (i, j, w) in r_sol:
        if w == t:
            week_pairs.append(f"Pattern {i} vs Pattern {j}")
    calendar_data[f"Week {t+1}"] = week_pairs

# Create and display the DataFrame
calendar_df = pd.DataFrame(calendar_data)
calendar_df

,Week 1,Week 2,Week 3,Week 4,Week 5
0,Pattern 1 vs Pattern 7,Pattern 1 vs Pattern 6,Pattern 1 vs Pattern 4,Pattern 1 vs Pattern 9,Pattern 1 vs Pattern 12
1,Pattern 4 vs Pattern 12,Pattern 4 vs Pattern 7,Pattern 6 vs Pattern 12,Pattern 4 vs Pattern 6,Pattern 4 vs Pattern 9
2,Pattern 6 vs Pattern 9,Pattern 9 vs Pattern 12,Pattern 7 vs Pattern 9,Pattern 7 vs Pattern 12,Pattern 6 vs Pattern 7


Checking Patterns for 'H' and 'A' Specifications

In [33]:
pattern_to_team = {team_numbers[team] : p for (team, p), value in z_sol.items() if value > 0.5}
weeks = {}

for (team1, team2, week), value in x_sol.items():
    if value == 1.0: 
        if week not in weeks:
            weeks[week] = []
        key = pattern_to_team[team_numbers[team1]]  
        if valid_patterns[key][week] == 'H':
            match = f"{team_numbers[team1]} vs {team_numbers[team2]}"
        else:
            match = f"{team_numbers[team2]} vs {team_numbers[team1]}"
        if match not in weeks[week]:
            weeks[week].append(match)




Displaying Final Output

In [34]:
# Sort the weeks to ensure correct order
sorted_weeks_example = sorted(weeks.keys())

# Find the maximum number of matches in a week to standardize row count
max_matches_example = max(len(matches) for matches in weeks.values())

# Create a DataFrame with rows as matches and columns as weeks
formatted_data_example = {
    f"Week {week}": weeks[week] + [None] * (max_matches_example - len(weeks[week]))
    for week in sorted_weeks_example
}

formatted_df_example = pd.DataFrame(formatted_data_example)
formatted_df_example

,Week 0,Week 1,Week 2,Week 3,Week 4
0,Team 1 vs Team 4,Team 1 vs Team 2,Team 3 vs Team 1,Team 5 vs Team 1,Team 1 vs Team 6
1,Team 2 vs Team 5,Team 4 vs Team 3,Team 6 vs Team 2,Team 2 vs Team 3,Team 4 vs Team 2
2,Team 3 vs Team 6,Team 5 vs Team 6,Team 4 vs Team 5,Team 6 vs Team 4,Team 5 vs Team 3
